In [1]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

Using TensorFlow backend.
/home/reorder/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/reorder/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/reorder/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/reorder/.local/lib/python3.6/site-packages/tensorflow/py

In [2]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',input_shape=(100,100,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(40, activation='sigmoid'))
    model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="binary_crossentropy",metrics=["accuracy"])
    return model

In [3]:
model = get_model()


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
model.load_weights("multilabel.h5")

In [9]:
import pandas as pd
attr_df = pd.read_csv("../list_attr_celeba.csv")
Labels =  [i for i in attr_df.columns]
Labels=Labels[1:]

In [10]:
Labels

['5_o_Clock_Shadow',
 'Arched_Eyebrows',
 'Attractive',
 'Bags_Under_Eyes',
 'Bald',
 'Bangs',
 'Big_Lips',
 'Big_Nose',
 'Black_Hair',
 'Blond_Hair',
 'Blurry',
 'Brown_Hair',
 'Bushy_Eyebrows',
 'Chubby',
 'Double_Chin',
 'Eyeglasses',
 'Goatee',
 'Gray_Hair',
 'Heavy_Makeup',
 'High_Cheekbones',
 'Male',
 'Mouth_Slightly_Open',
 'Mustache',
 'Narrow_Eyes',
 'No_Beard',
 'Oval_Face',
 'Pale_Skin',
 'Pointy_Nose',
 'Receding_Hairline',
 'Rosy_Cheeks',
 'Sideburns',
 'Smiling',
 'Straight_Hair',
 'Wavy_Hair',
 'Wearing_Earrings',
 'Wearing_Hat',
 'Wearing_Lipstick',
 'Wearing_Necklace',
 'Wearing_Necktie',
 'Young']

In [12]:
datagen=ImageDataGenerator(rescale=1./255.)
test_generator = datagen.flow_from_dataframe(dataframe=attr_df[:2],directory="/home/reorder/feature_extraction/archive/img_align_celeba/img_align_celeba/",x_col="image_id",y_col=Labels,batch_size=1,seed=42,shuffle=False,class_mode=None,target_size=(100,100))

Found 2 validated image filenames.


In [14]:
pred = model.predict_generator(test_generator,verbose=1)

2/2 [==============================] - 0s 12ms/step


In [15]:
pred_bool = (pred >0.5)

In [16]:
pred_bool

array([[False,  True,  True, False, False, False, False, False, False,
        False, False,  True, False, False, False, False, False, False,
         True,  True, False,  True, False, False,  True, False, False,
         True, False, False, False,  True, False,  True, False, False,
         True, False, False,  True],
       [False, False,  True,  True, False, False, False, False, False,
        False, False,  True, False, False, False, False, False, False,
        False,  True, False,  True, False, False,  True, False, False,
        False, False, False, False,  True, False, False, False, False,
        False, False, False,  True]])

In [6]:
from PIL import Image
import numpy as np
from skimage import transform
import requests
def load(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (100, 100, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

url = "https://images.pexels.com/photos/1391498/pexels-photo-1391498.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500"
img=requests.get(url, stream=True).raw
image = load(img)
pred = model.predict(image)
pred_bool = (pred >0.5)
print(pred_bool)


[[False False False False False False False False False False False False
  False False False False False False False False  True False False False
   True False False False False False False False False False False False
  False False False False]]


In [ ]:
import cv2
data = cv2.imread('/home/reorder/feature_extraction/archive/img_align_celeba/img_align_celeba/018946.jpg')
cv2.imshow("window",data)
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [11]:
from itertools import compress
list(compress(Labels, pred_bool[0]))

['Male', 'No_Beard']

In [10]:
import requests
url = "https://images.pexels.com/photos/1391498/pexels-photo-1391498.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500"
img=requests.get(url, stream=True).raw

In [11]:
img

In [12]:
np_image = Image.open(img)

In [17]:
path = str(input("Enter the Image Path"))
print("Path : ",path)
image = load(path)
pred = model.predict(image)
pred_bool = (pred >0.5)

Labels = ['5_o_Clock_Shadow','Arched_Eyebrows','Attractive','Bags_Under_Eyes','Bald','Bangs','Big_Lips','Big_Nose','Black_Hair','Blond_Hair','Blurry','Brown_Hair','Bushy_Eyebrows','Chubby','Double_Chin','Eyeglasses','Goatee','Gray_Hair','Heavy_Makeup','High_Cheekbones','Male','Mouth_Slightly_Open','Mustache','Narrow_Eyes','No_Beard','Oval_Face','Pale_Skin','Pointy_Nose','Receding_Hairline','Rosy_Cheeks','Sideburns','Smiling','Straight_Hair','Wavy_Hair','Wearing_Earrings','Wearing_Hat','Wearing_Lipstick','Wearing_Necklace','Wearing_Necktie','Young']
output = list(compress(Labels, pred_bool[0]))
print("Output : ",output)

Enter the Image Path/home/reorder/Desktop/index.jpeg
Path :  /home/reorder/Desktop/index.jpeg
Output :  ['Blurry', 'Male', 'No_Beard', 'Young']
